<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [1]:
# mount drive

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir spacy_ner_data

In [ ]:
import json

import random

from sklearn.model_selection import train_test_split

import spacy

from spacy.tokens import DocBin



# Load JSON data

with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/200_resumes_annotated.json', "r") as f:

    data = json.load(f)



def remove_overlapping_entities(entities):

    """Remove overlapping entities from the list."""

    entities = sorted(entities, key=lambda x: x[0])  # Sort by start position

    non_overlapping = []

    last_end = -1

    for start, end, label in entities:

        if start >= last_end:  # Only add if there's no overlap with the previous entity

            non_overlapping.append((start, end, label))

            last_end = end

    return non_overlapping



# Function to convert JSON data to Spacy's DocBin format

def convert_to_spacy_format(data):

    nlp = spacy.blank("en")  # Load a blank Spacy model

    doc_bin = DocBin()  # Container for our docs



    for item in data:

        text = item['data']['Text']  # Full document text

        entities = []



        for annotation in item['annotations'][0]['result']:

            start = annotation['value']['start']

            end = annotation['value']['end']

            label = annotation['value']['labels'][0]  # Entity label

            entities.append((start, end, label))



        entities = remove_overlapping_entities(entities)  # Remove overlapping entities

        # Create a Spacy doc and add entities to it

        doc = nlp.make_doc(text)

        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]

        # Filter out None spans if Spacy can't align the character indices with tokens

        spans = [span for span in spans if span is not None]

        doc.ents = spans  # Assign entities to the doc

        doc_bin.add(doc)



    return doc_bin



# Split data into train and test sets

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)



# Convert train and test sets to Spacy format

train_doc_bin = convert_to_spacy_format(train_data)

test_doc_bin = convert_to_spacy_format(test_data)



# Save the train and test data to .spacy files

train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")

test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")

## Spacy NER

In [ ]:
# create base_config.cfg and paste the config generated from spacy widget

# update train and test file path

!touch base_config.cfg

In [ ]:
# generate config.cfg from base_config.cfg

!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# train model using hyperparameters set in config.cfg

# save trained model in spacy-output/ dir

!python -m spacy train config.cfg --output ./spacy_output

!cp -r ./spacy_output /content/drive/MyDrive/FYP/Implementation/

In [ ]:
# evaluate trained model performance

# store output and visualization into result/ dir

!python -m spacy evaluate spacy_output/model-best spacy_ner_data/test_data.spacy -dp spacy_output

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   51.12 
NER R   41.26 
NER F   45.66 
SPEED   2395  


=============================== NER (per type) ===============================

                P       R       F
NAME        89.66   78.79   83.87
JOB         72.00   32.43   44.72
DEG         62.16   63.89   63.01
UNI         38.89   34.15   36.36
EMAIL       63.33   95.00   76.00
LOC         39.39   31.71   35.14
WORK PER    75.45   83.00   79.05
COMPANY     28.42   36.49   31.95
SKILL       40.96   28.96   33.93
PHONE       89.66   83.87   86.67
STUDY PER   65.62   58.33   61.76

<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__main__.p

In [ ]:
# make prediction

import spacy

resume_text = '''

John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.



John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'

'''

nlp = spacy.load("spacy-output/model-best")

doc = nlp(resume_text.lower())



print(doc.ents)



for ent in doc.ents:

    print(f"{ent.text}: {ent.label_}")

(john doe, in los, (555) 123-4567, john.doe@example.com, john is, aws, restful apis, master of science, bachelor of science in information technology, python, aws, azure, git, docker)
john doe: NAME
in los: LOC
(555) 123-4567: PHONE
john.doe@example.com: EMAIL
john is: NAME
aws: SKILL
restful apis: SKILL
master of science: DEG
bachelor of science in information technology: DEG
python: SKILL
aws: SKILL
azure: SKILL
git: SKILL
docker: SKILL


In [ ]:
from spacy import displacy

displacy.render(doc, style="ent", jupyter=True)

## Flair NER

In [1]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 64.7 MB/s eta 0:00:00:00:010:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=0d123d3537fbbbedb2d35727d4461b744f3d7bdf6b48f46a278c4766ee11a161
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
  Created wheel for pptree: filename=pptree-3.1-py3-none-any.whl size=4609 sha256=59b5dff4e480bef279e469b0b090877ad013a7eb6e296b0c8900788b55b7bf12
  Stored in directory: /root/.cache/pip/whee

In [ ]:
# convert spacy data into flair data

import spacy

from spacy.tokens import DocBin

import os



def convert_spacy_to_flair(input_file, output_file):

    """

    Convert SpaCy binary format to Flair's CoNLL format.



    Args:

        input_file (str): Path to SpaCy binary file (.spacy)

        output_file (str): Path to output file for Flair format

    """

    # Load spaCy model

    nlp = spacy.blank("en")



    # Load the DocBin

    doc_bin = DocBin().from_disk(input_file)

    docs = list(doc_bin.get_docs(nlp.vocab))



    with open(output_file, 'w', encoding='utf-8') as f:

        for doc in docs:

            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]



            # Write tokens in CoNLL format

            for token in tokens:

                text, iob, ent_type = token



                # Convert spaCy IOB to CoNLL format

                if iob == 'O':

                    tag = 'O'

                else:

                    tag = f'{iob}-{ent_type}' if ent_type else 'O'



                # Write line: token and NER tag

                f.write(f'{text} {tag}\n')



            # Empty line between sentences

            f.write('\n')



def convert_spacy_json_to_flair(input_file, output_file):

    """

    Convert SpaCy JSON format to Flair's CoNLL format.



    Args:

        input_file (str): Path to JSON file with SpaCy annotations

        output_file (str): Path to output file for Flair format

    """

    import json



    nlp = spacy.blank("en")



    with open(input_file, 'r', encoding='utf-8') as f:

        training_data = json.load(f)



    with open(output_file, 'w', encoding='utf-8') as f:

        for example in training_data:

            text = example['text']

            ents = example.get('entities', [])



            # Create a spaCy doc

            doc = nlp(text)



            # Add entities to doc

            spans = []

            for start, end, label in ents:

                span = doc.char_span(start, end, label=label)

                if span is not None:

                    spans.append(span)

            doc.ents = spans



            # Convert to CoNLL format

            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]



            for token in tokens:

                text, iob, ent_type = token

                if iob == 'O':

                    tag = 'O'

                else:

                    tag = f'{iob}-{ent_type}' if ent_type else 'O'

                f.write(f'{text} {tag}\n')



            f.write('\n')



# Example usage for JSON format

flair_train_json = "flair_train.txt"

flair_test_json = "flair_test.txt"



convert_spacy_to_flair('/content/spacy_ner_data/train_data.spacy', flair_train_json)

convert_spacy_to_flair('/content/spacy_ner_data/test_data.spacy', flair_test_json)

FileNotFoundError: [Errno 2] No such file or directory: '/content/spacy_ner_data/train_data.spacy'

In [ ]:
# convert spacy data into flair data

import spacy

from spacy.training import Corpus



!python -m spacy download de_core_news_sm

nlp = spacy.load("de_core_news_sm")

corpus = Corpus("/content/spacy_ner_data/test_data.spacy")



data = corpus(nlp)



# Flair supports BIO and BIOES, see https://github.com/flairNLP/flair/issues/875

def rename_biluo_to_bioes(old_tag):

    new_tag = ""

    try:

        if old_tag.startswith("L"):

            new_tag = "E" + old_tag[1:]

        elif old_tag.startswith("U"):

            new_tag = "S" + old_tag[1:]

        else:

            new_tag = old_tag

    except:

        pass

    return new_tag





def generate_corpus():

    corpus = []

    n_ex = 0

    for example in data:

        n_ex += 1

        text = example.text

        doc = nlp(text)

        tags = example.get_aligned_ner()

        # Check if it's an empty list of NER tags.

        if None in tags:

            pass

        else:

            new_tags = [rename_biluo_to_bioes(tag) for tag in tags]

            for token, tag in zip(doc,new_tags):

                row = token.text +' '+ token.pos_ +' ' +tag + '\n'

                corpus.append(row)

            corpus.append('\n')

    return corpus



def write_file(filepath):

    with open(filepath, 'w', encoding='utf-8') as f:

        corpus = generate_corpus()

        f.writelines(corpus)



def main():

    write_file('flair_test.txt')



main()

  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
# convert json into flair data

import json

import random

from typing import List, Dict, Tuple

import spacy

from collections import defaultdict



class NERConverter:

    def __init__(self):

        self.nlp = spacy.load("en_core_web_sm")



    def get_bioes_label(self, token_index: int, entity_length: int, current_position: int, label: str) -> str:

        """

        Convert to BIOES format

        - S-: Single token entity

        - B-: Beginning of multi-token entity

        - I-: Inside of multi-token entity

        - E-: End of multi-token entity

        - O: Outside

        """

        if entity_length == 1:

            return f'S-{label}'

        if current_position == 0:

            return f'B-{label}'

        if current_position == entity_length - 1:

            return f'E-{label}'

        return f'I-{label}'



    def convert_to_bioes_format(self, json_data: List[dict]) -> List[List[Tuple[str, str]]]:

        """Convert JSON annotations to BIOES format."""

        all_sentences = []



        for item in json_data:

            text = item['data']['Text']

            doc = self.nlp(text)



            # Initialize character-level labels

            char_labels = ['O'] * len(text)



            # First pass: identify entity boundaries and lengths

            entity_spans = []

            if item['annotations'] and len(item['annotations']) > 0:

                for ann in item['annotations'][0]['result']:

                    if 'value' in ann:

                        start = ann['value']['start']

                        end = ann['value']['end']

                        label = ann['value']['labels'][0]

                        entity_spans.append((start, end, label))



            # Sort spans by start position

            entity_spans.sort(key=lambda x: x[0])



            # Second pass: apply BIOES labels

            for start, end, label in entity_spans:

                # Get tokens that are part of this entity

                entity_text = text[start:end]

                entity_doc = self.nlp(entity_text)

                entity_length = len([token for token in entity_doc if not token.is_space])



                # Set labels for the entire span

                current_token_idx = 0

                for i in range(start, end):

                    if i == start or text[i-1].isspace():

                        char_labels[i] = self.get_bioes_label(i, entity_length, current_token_idx, label)

                        current_token_idx += 1

                    else:

                        char_labels[i] = char_labels[i-1]



            # Convert to token-level labels

            current_sentence = []

            for sent in doc.sents:

                for token in sent:

                    # Get the most common label for the token's characters

                    token_chars_labels = char_labels[token.idx:token.idx + len(token.text)]

                    label_counts = defaultdict(int)

                    for char_label in token_chars_labels:

                        label_counts[char_label] += 1



                    token_label = max(label_counts.items(), key=lambda x: x[1])[0]

                    current_sentence.append((token.text, token_label))



                if current_sentence:

                    all_sentences.append(current_sentence)

                    current_sentence = []



        return all_sentences



    def write_flair_file(self, sentences: List[List[Tuple[str, str]]], filename: str):

        """Write sentences in BIOES format to file."""

        with open(filename, 'w', encoding='utf-8') as f:

            for sentence in sentences:

                for token, label in sentence:

                    f.write(f'{token} {label}\n')

                f.write('\n')



    def convert_and_split(self, json_data: List[dict], train_file: str, test_file: str, test_ratio: float = 0.2):

        """Convert JSON to BIOES format and split into train/test sets."""

        all_sentences = self.convert_to_bioes_format(json_data)



        # Shuffle and split

        random.shuffle(all_sentences)

        split_idx = int(len(all_sentences) * (1 - test_ratio))



        train_sentences = all_sentences[:split_idx]

        test_sentences = all_sentences[split_idx:]



        # Write to files

        self.write_flair_file(train_sentences, train_file)

        self.write_flair_file(test_sentences, test_file)



        return len(train_sentences), len(test_sentences)



def main():

    # Load JSON data

    with open('/kaggle/input/resume-dataset/200_resumes_annotated.json', 'r', encoding='utf-8') as f:

        json_data = json.load(f)



    # Convert and split data

    converter = NERConverter()

    train_count, test_count = converter.convert_and_split(

        json_data,

        train_file='flair_train.txt',

        test_file='flair_test.txt',

        test_ratio=0.2

    )



    print(f'Created {train_count} training sentences and {test_count} test sentences')



main()

Created 298 training sentences and 75 test sentences


In [4]:
from flair.data import Corpus

from flair.datasets import ColumnCorpus



# Define columns for CoNLL (0: word, 1: label)

columns = {0: 'text', 1: 'ner'}



# Set data folder and file names

data_folder = './'

train_file = '/kaggle/working/flair_test.txt'

test_file = '/kaggle/working/flair_test.txt'



# Load the corpus

corpus: Corpus = ColumnCorpus(data_folder, columns,

                              train_file=train_file,

                              test_file=test_file,

                              dev_file=None)

2024-11-06 13:50:53,363 Reading data from .
2024-11-06 13:50:53,365 Train: /kaggle/working/flair_test.txt
2024-11-06 13:50:53,366 Dev: None
2024-11-06 13:50:53,367 Test: /kaggle/working/flair_test.txt
2024-11-06 13:50:54,194 No dev split found. Using 10% (i.e. 8 samples) of the train split as dev data


In [5]:
tag_dictionary = corpus.make_label_dictionary(label_type='ner')

print("Labels:", tag_dictionary.get_items())

2024-11-06 13:50:57,494 Computing label dictionary. Progress:


0it [00:00, ?it/s]
67it [00:00, 9716.42it/s]

2024-11-06 13:50:57,529 Dictionary created for label 'ner' with 11 values: SKILL (seen 435 times), JOB (seen 116 times), WORK (seen 97 times), COMPANY (seen 70 times), LOC (seen 61 times), NAME (seen 37 times), PHONE (seen 37 times), UNI (seen 36 times), DEG (seen 30 times), EMAIL (seen 28 times), STUDY (seen 26 times)
Labels: ['SKILL', 'JOB', 'WORK', 'COMPANY', 'LOC', 'NAME', 'PHONE', 'UNI', 'DEG', 'EMAIL', 'STUDY']


In [6]:
from collections import Counter



def count_labels(file_path):

    with open(file_path, 'r') as file:

        labels = [line.split()[-1] for line in file if line.strip()]

    return Counter(labels)



print("Train label distribution:", count_labels('flair_train.txt'))

print("Test label distribution:", count_labels('flair_test.txt'))

Train label distribution: Counter({'O': 73362, 'S-SKILL': 1509, 'PER': 1232, 'B-SKILL': 1008, 'E-SKILL': 1004, 'E-JOB': 543, 'B-JOB': 533, 'E-COMPANY': 365, 'B-COMPANY': 361, 'I-JOB': 302, 'I-COMPANY': 273, 'I-DEG': 265, 'I-SKILL': 187, 'E-UNI': 172, 'B-UNI': 171, 'S-LOC': 166, 'E-DEG': 165, 'B-DEG': 164, 'B-NAME': 150, 'E-NAME': 150, 'I-UNI': 141, 'E-PHONE': 124, 'B-PHONE': 122, 'S-EMAIL': 118, 'B-LOC': 101, 'E-LOC': 101, 'I-PHONE': 84, 'S-JOB': 60, 'S-COMPANY': 59, 'S-PHONE': 21, 'I-NAME': 9, 'S-DEG': 7, 'I-LOC': 5, 'S-UNI': 2, 'B-EMAIL': 1, 'E-EMAIL': 1})
Test label distribution: Counter({'O': 15731, 'S-SKILL': 282, 'PER': 279, 'B-SKILL': 205, 'E-SKILL': 205, 'E-JOB': 116, 'B-JOB': 114, 'I-DEG': 69, 'E-COMPANY': 62, 'B-COMPANY': 61, 'I-JOB': 55, 'I-SKILL': 49, 'B-UNI': 43, 'E-UNI': 43, 'I-COMPANY': 42, 'B-NAME': 41, 'E-NAME': 41, 'S-LOC': 39, 'B-DEG': 37, 'E-DEG': 37, 'B-PHONE': 33, 'E-PHONE': 33, 'I-UNI': 30, 'S-EMAIL': 30, 'B-LOC': 27, 'E-LOC': 27, 'I-PHONE': 22, 'S-COMPANY': 20, 

In [7]:
# create NER tagger

from flair.embeddings import WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings, FlairEmbeddings

from flair.models import SequenceTagger



# using LSTM-CRF on top of frozen embeddings

# combine flair and glove embeddings

# embeddings = StackedEmbeddings([

#                 WordEmbeddings('glove'),

#                 FlairEmbeddings('news-forward'),

#                 FlairEmbeddings('news-backward'),

#             ])

# tagger = SequenceTagger(hidden_size=256,

#                          embeddings=embeddings,

#                          tag_dictionary=tag_dictionary,

#                          tag_type='ner',

#                          use_crf=True,

#                          tag_format = 'BIOES')



# using transformer embedding

# embeddings = TransformerWordEmbeddings('bert-base-uncased',

#                                       fine_tune=True,

#                                       layers='-1',

#                                       subtoken_pooling='first')

embeddings = TransformerWordEmbeddings(

    'roberta-base',  # or 'bert-base-uncased'

    fine_tune=True,

    layers='-1,-2,-3,-4',  # Use last 4 layers

    subtoken_pooling='first',

    allow_long_sentences=True

)

tagger = SequenceTagger(hidden_size=256,

                         embeddings=embeddings,

                         tag_dictionary=tag_dictionary,

                         tag_type='ner',

                         use_crf=False,

                         use_rnn=False,

                         reproject_embeddings=False,

                         tag_format = 'BIOES')

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

2024-11-06 13:51:16,469 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, S-STUDY, B-STUDY, E-STUDY, I-STUDY


In [8]:
# train flair ner model

from flair.trainers import ModelTrainer

from flair.training_utils import EvaluationMetric



trainer = ModelTrainer(tagger, corpus)



trainer.train(

    base_path='flair_output/',

    learning_rate=5.0e-5,

    mini_batch_size=16,

    max_epochs=50,

    train_with_dev=False

)

!cp -r ./flair_output /kaggle/working/

2024-11-06 13:51:24,215 ----------------------------------------------------------------------------------------------------
2024-11-06 13:51:24,217 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50266, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features

/opt/conda/lib/python3.10/site-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-11-06 13:51:26,829 epoch 1 - iter 1/5 - loss 3.74054938 - time (sec): 2.59 - samples/sec: 1316.54 - lr: 0.300000 - momentum: 0.000000
2024-11-06 13:51:27,598 epoch 1 - iter 2/5 - loss 3.36812881 - time (sec): 3.35 - samples/sec: 2171.90 - lr: 0.300000 - momentum: 0.000000
2024-11-06 13:51:29,493 epoch 1 - iter 3/5 - loss 2.56930247 - time (sec): 5.25 - samples/sec: 2142.94 - lr: 0.300000 - momentum: 0.000000
2024-11-06 13:51:31,534 epoch 1 - iter 4/5 - loss 2.62421327 - time (sec): 7.29 - samples/sec: 2085.58 - lr: 0.300000 - momentum: 0.000000
2024-11-06 13:51:32,861 epoch 1 - iter 5/5 - loss 2.64770448 - time (sec): 8.62 - samples/sec: 1832.90 - lr: 0.300000 - momentum: 0.000000
2024-11-06 13:51:32,862 ----------------------------------------------------------------------------------------------------
2024-11-06 13:51:32,864 EPOCH 1 done: loss 2.6477 - lr: 0.300000


100%|██████████| 1/1 [00:00<00:00,  1.03it/s]

2024-11-06 13:51:33,870 DEV : loss 2.8290953636169434 - f1-score (micro avg)  0.031
2024-11-06 13:51:33,876  - 0 epochs without improvement
2024-11-06 13:51:33,877 saving best model


...

2024-11-06 14:00:22,548 Done.
2024-11-06 14:00:22,549 ----------------------------------------------------------------------------------------------------
2024-11-06 14:00:22,552 Loading model from best epoch ...
2024-11-06 14:00:25,740 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, S-STUDY, B-STUDY, E-STUDY, I-STUDY


100%|██████████| 2/2 [00:08<00:00,  4.43s/it]

2024-11-06 14:00:34,924 
Results:
- F-score (micro) 0.0298
- F-score (macro) 0.0028
- Accuracy 0.0153

By class:
              precision    recall  f1-score   support

       SKILL     0.0158    0.5791    0.0308       487
         JOB     0.0000    0.0000    0.0000       130
        WORK     0.0000    0.0000    0.0000       111
     COMPANY     0.0000    0.0000    0.0000        82
         LOC     0.0000    0.0000    0.0000        66
         UNI     0.0000    0.0000    0.0000        43
        NAME     0.0000    0.0000    0.0000        41
       PHONE     0.0000    0.0000    0.0000        41
         DEG     0.0000    0.0000    0.0000        37
       STUDY     0.0000    0.0000    0.0000        34
       EMAIL     0.0000    0.0000    0.0000        30

   micro avg     0.0158    0.2559    0.0298      1102
   macro avg     0.0014    0.0526    0.0028      1102
weighted avg     0.0070    0.2559    0.0136      1102

2024-11-06 14:00:34,925 --------------------------------------------------


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


cp: './flair_output' and '/kaggle/working/flair_output' are the same file


In [10]:
# evaluate model

from flair.data import Corpus

from flair.datasets import ColumnCorpus

from flair.models import SequenceTagger

from flair.trainers import ModelTrainer



# Load the trained model

model = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/final-model.pt')



# Evaluate the model on the test set

result = model.evaluate(corpus.test, gold_label_type='ner', mini_batch_size=32)



# Print the results

# print("Evaluation Loss:", eval_loss)

print(result.detailed_results)  # print the precision, recall, and F1-score per entity type

2024-11-06 10:57:38,582 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-DEG, B-DEG, E-DEG, I-DEG, S-UNI, B-UNI, E-UNI, I-UNI, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL


100%|██████████| 3/3 [00:03<00:00,  1.29s/it]


Results:
- F-score (micro) 0.563
- F-score (macro) 0.7187
- Accuracy 0.3962

By class:
              precision    recall  f1-score   support

       SKILL     0.3408    0.3862    0.3621       593
         JOB     0.5730    0.7020    0.6310       151
        WORK     0.8188    0.9457    0.8777       129
     COMPANY     0.5172    0.5714    0.5430       105
         LOC     0.5701    0.7176    0.6354        85
         DEG     0.5862    0.7907    0.6733        43
         UNI     0.5660    0.6667    0.6122        45
        NAME     0.9388    0.9787    0.9583        47
       PHONE     1.0000    0.9787    0.9892        47
       EMAIL     0.8222    0.9250    0.8706        40
       STUDY     0.8286    0.6905    0.7532        42

   micro avg     0.5281    0.6029    0.5630      1327
   macro avg     0.6874    0.7594    0.7187      1327
weighted avg     0.5324    0.6029    0.5642      1327



In [11]:
from flair.models import SequenceTagger

from flair.data import Sentence

import spacy

from spacy import displacy



# Load your trained Flair NER model

tagger = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt')



resume_text = '''

John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.



John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'

'''



# Step 1: Predict entities using Flair

sentence = Sentence(resume_text)

tagger.predict(sentence)



# Step 2: Convert Flair predictions to spaCy doc format

# Initialize a blank spaCy NLP pipeline

nlp = spacy.blank("en")

doc = nlp(resume_text)



# Extract entities from Flair prediction and convert to spaCy format

ents = []

for entity in sentence.get_spans('ner'):

    start, end = entity.start_position, entity.end_position

    label = entity.tag

    span = doc.char_span(start, end, label=label)

    if span is not None:

        ents.append(span)



# Set the entities in the spaCy doc

doc.ents = ents



# Step 3: Visualize using displacy

# Display in Jupyter or a web page

displacy.render(doc, style="ent", jupyter=True)


2024-11-06 10:57:55,781 SequenceTagger predicts: Dictionary with 47 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, <START>, <STOP>


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [12]:
from flair.models import SequenceTagger

from flair.data import Sentence



# Load the pretrained NER model

tagger = SequenceTagger.load("/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt")

# Example text

text = "Apple is looking at buying U.K. startup for $1 billion."



# Create a Sentence object

sentence = Sentence(resume_text)



# Predict entities

tagger.predict(sentence)



# Print the detected entities

for entity in sentence.get_spans("ner"):

    print(f"Entity: {entity.text}, Type: {entity.get_label('ner').value}, Confidence: {entity.score}")


2024-11-06 10:58:17,528 SequenceTagger predicts: Dictionary with 47 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, <START>, <STOP>
Entity: Doe lives, Type: NAME, Confidence: 0.16702505946159363
Entity: 1234 Elm, Type: NAME, Confidence: 0.11599351465702057
Entity: Los, Type: DEG, Confidence: 0.18544061481952667
Entity: 90001, Type: DEG, Confidence: 0.14748063683509827
Entity: 555, Type: DEG, Confidence: 0.11017131805419922
Entity: 123-4567, Type: DEG, Confidence: 0.11569119244813919
Entity: via, Type: DEG, Confidence: 0.20629706978797913
Entity: at, Type: EMAIL, Confidence: 0.19365909695625305
Entity: john.doe, Type: JOB, Confidence: 0.10587572306394577
Entity: @, Ty